In [58]:
import pickle

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split


device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [59]:
data = pd.read_csv("drug200.csv")
data

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [60]:
X_data, y_data = data.drop(columns=["Drug"]), data["Drug"]

x_shape = X_data.shape[1]
num_classes = y_data.nunique()

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, stratify=y_data
)

In [62]:
from typing import Iterable


class Encoder:
    def __init__(self, column_list: Iterable):
        self.column_list = column_list

    def encoder_data(self, data: pd.DataFrame, mode: str):
        le = LabelEncoder()
        for column in self.column_list:
            if mode != "test":
                data[column] = le.fit_transform(data[column])
                with open(f"le_{column}.pkl", "wb") as le_dump_file:
                    pickle.dump(le, le_dump_file)
            else:
                with open(f"le_{column}.pkl", "rb") as le_dump_file:
                    le = pickle.load(le_dump_file)
                data[column] = le.transform(data[column])
        return np.array(data)

    def encoder_target(self, data: pd.DataFrame, inverse: bool = False):
        if inverse:
            with open("le_target.pkl", "rb") as le_load_file:
                le = pickle.load(le_load_file)
            data = le.inverse_transform(data)
        else:
            le = LabelEncoder()
            data = le.fit_transform(data)
            with open("le_target.pkl", "wb") as le_dump_file:
                pickle.dump(le, le_dump_file)
        return np.array(data)

In [63]:
class My_dataset(Dataset):
    def __init__(
        self, X_data: pd.DataFrame, Y_data: pd.DataFrame, encoder: Encoder, mode: str
    ):
        self.x = torch.tensor(encoder.encoder_data(X_data, mode), dtype=torch.float32)
        self.mode = mode
        if self.mode != "test":
            self.y = torch.tensor(encoder.encoder_target(Y_data), dtype=torch.int64)

    def __len__(self):
        return self.x.size(0)

    def __getitem__(self, idx: int):
        if self.mode != "test":
            return self.x[idx, :], self.y[idx]
        else:
            return self.x[idx, :]

In [64]:
column_list = ["Sex", "BP", "Cholesterol"]
encoder = Encoder(column_list)

In [65]:
train_data = My_dataset(X_train, y_train, encoder, mode="train")
test_data = My_dataset(X_test, y_test, encoder, mode="test")

train_subset, val_subset = random_split(
    train_data, [int(len(train_data) * 0.8), int(len(train_data) * 0.2)]
)

train_loader = DataLoader(dataset=train_subset, shuffle=True, batch_size=16)
val_loader = DataLoader(dataset=val_subset, shuffle=False, batch_size=16)
test_loader = DataLoader(dataset=test_data, shuffle=False, batch_size=16)

In [66]:
class ClassificationNetwork(nn.Module):
    def __init__(self, input_shape: int, num_classes: int):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_shape, num_classes), nn.LogSoftmax(dim=-1)
        )

    def forward(self, input):
        out = self.model(input)
        return out

NLLloss SoftMax LogSoftMax

In [67]:
model = ClassificationNetwork(x_shape, num_classes).to(device)

number_epoch = 100
learning_rate = 0.01
min_val_loss = 10000

train_loss_list = []
val_loss_list = []
train_accuracy_list = []
val_accuracy_list = []

opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

for epoch in range(number_epoch):
    sum_train_loss = 0
    sum_val_loss = 0
    sum_correct_train_prediction = 0
    sum_correct_val_prediction = 0

    model.train()
    for train_data, train_label in train_loader:
        train_preds = model(train_data.to(device))
        train_loss = loss_func(train_preds, train_label.to(device))

        opt.zero_grad()
        train_loss.backward()
        opt.step()

        sum_train_loss += train_loss.item() * train_data.size(0)

        _, train_prediction = torch.max(train_preds, 1)
        sum_correct_train_prediction += torch.sum(
            train_prediction == train_label.to(device)
        ).item()

    model.eval()
    for val_data, val_label in val_loader:
        val_preds = model(val_data.to(device))
        val_loss = loss_func(val_preds, val_label.to(device))
        sum_val_loss += val_loss.item() * val_data.size(0)

        _, val_prediction = torch.max(val_preds, 1)
        sum_correct_val_prediction += torch.sum(
            val_prediction == val_label.to(device)
        ).item()

    if val_loss < min_val_loss:
        best_model = torch.save(model.state_dict(), "model.pt")
        min_val_loss = val_loss

    mean_train_loss = sum_train_loss / len(train_loader.sampler)
    mean_val_loss = sum_val_loss / len(val_loader.sampler)
    if epoch % 5 == 0:
        print(
            f"Epoch {epoch}:   Train loss: {mean_train_loss}    Val loss: {mean_val_loss}"
        )

    train_loss_list.append(mean_train_loss)
    val_loss_list.append(mean_val_loss)
    train_accuracy_list.append(sum_correct_train_prediction / len(train_loader.sampler))
    val_accuracy_list.append(sum_correct_val_prediction / len(val_loader.sampler))

Epoch 0:   Train loss: 8.462640106678009    Val loss: 4.905186176300049
Epoch 5:   Train loss: 1.686147227883339    Val loss: 1.1139031052589417
Epoch 10:   Train loss: 0.8279839530587196    Val loss: 0.7851267755031586
Epoch 15:   Train loss: 0.719148188829422    Val loss: 0.6833127737045288
Epoch 20:   Train loss: 0.6659297049045563    Val loss: 0.6373114585876465
Epoch 25:   Train loss: 0.6086044907569885    Val loss: 0.6275662928819656
Epoch 30:   Train loss: 0.5956403948366642    Val loss: 0.6651550233364105
Epoch 35:   Train loss: 0.5479634664952755    Val loss: 0.5965103507041931
Epoch 40:   Train loss: 0.5137886255979538    Val loss: 0.5340329855680466
Epoch 45:   Train loss: 0.5011645033955574    Val loss: 0.5264345556497574
Epoch 50:   Train loss: 0.4723559580743313    Val loss: 0.5315114557743073
Epoch 55:   Train loss: 0.47158458828926086    Val loss: 0.5121899247169495
Epoch 60:   Train loss: 0.4555029645562172    Val loss: 0.5348613113164902
Epoch 65:   Train loss: 0.4210

In [73]:
model.load_state_dict(torch.load("model.pt"))
sum_correct_test_prediction = 0

test_preds_list = []
test_accuracy_list = []

model.eval()
for test_data in test_loader:
    test_preds_list.append(model.forward(test_data.to(device)))

test_preds = torch.cat(test_preds_list, dim=0)

test_label = torch.tensor(encoder.encoder_target(y_test))

_, prediction = torch.max(test_preds, 1)
sum_correct_test_prediction += torch.sum(prediction == test_label.to(device)).item()

print(f"Accuracy: {sum_correct_test_prediction / len(test_loader.sampler)}")

Accuracy: 0.9


In [75]:
print(y_test.index)

95     drugX
32     drugX
35     drugX
172    DrugY
5      drugX
52     DrugY
120    DrugY
133    DrugY
168    DrugY
186    drugB
41     drugB
60     DrugY
66     drugA
131    DrugY
146    drugX
136    drugB
100    drugA
140    drugA
11     DrugY
192    DrugY
106    drugX
2      drugC
50     DrugY
59     DrugY
42     DrugY
198    drugX
98     DrugY
155    drugC
119    DrugY
38     drugX
159    drugX
194    DrugY
111    drugX
71     DrugY
143    DrugY
27     drugX
130    DrugY
76     drugA
147    drugA
84     drugC
Name: Drug, dtype: object
